<a href="https://colab.research.google.com/github/ksr1710/Ejercicios-basicos-html-y-js/blob/main/AppBESankey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from matplotlib.ticker import FuncFormatter

import plotly.graph_objects as go
import plotly.express as px

import itertools
import warnings

In [8]:
Base = pd.read_excel('/content/sanky.xlsx' )

In [16]:
df= Base.copy()
df =df.round(1)

In [17]:
df

,Categoria,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10
0,Estabilidad de la plataforma,43.5,34.1,26.6,34.1,36.1,38.7,43.9,39.0,40.4
1,Manejo de la solicitud y/o Inconformidad,13.0,19.8,25.3,13.4,15.7,18.9,15.8,15.3,15.4
2,Menu e interfaz,11.7,11.9,11.4,12.2,14.5,11.7,8.8,13.6,15.4
3,Costos,9.7,12.7,16.5,13.4,9.6,9.9,10.5,8.5,7.7
4,Felicitaciones,5.8,4.8,3.8,6.1,4.8,4.5,4.4,6.8,7.7
5,call_center,1.3,1.6,5.1,2.4,0.0,3.6,6.1,5.1,0.0
6,Seguridad,3.2,1.6,1.3,2.4,1.2,0.0,0.9,5.1,5.8
7,Falla_trx,3.2,3.2,0.0,1.2,1.2,3.6,3.5,0.0,0.0
8,InformaciÃ³n - ComunicaciÃ³n,1.3,1.6,0.0,2.4,3.6,1.8,0.9,3.4,5.8
9,oficina,2.6,1.6,1.3,2.4,3.6,1.8,0.0,0.0,0.0


In [18]:
listado_colores = ['#F45D5D',
                   '#808080',
                   '#DEB887',
                   '#AAFF32',
                   '#029386',
                   '#C875C4',
                   '#FAC205',
                   '#FF81C0',
                   '#FF00FF',
                   '#C5C9C7',
                   '#6E750E',
                   '#DDA00D',
                   '#FBDD7E',
                   '#E9967A',
                   '#00FFFF',
                   '#7FFFD4',
                   '#F5F5DC']

colors = {}

for val, color in zip(df.Categoria, listado_colores):
    colors[val] = color

colors

{'Estabilidad de la plataforma': '#F45D5D',
 'Manejo de la solicitud y/o Inconformidad': '#808080',
 'Menu e interfaz': '#DEB887',
 'Costos': '#AAFF32',
 'Felicitaciones': '#029386',
 'call_center': '#C875C4',
 'Seguridad': '#FAC205',
 'Falla_trx': '#FF81C0',
 'InformaciÃ³n - ComunicaciÃ³n': '#FF00FF',
 'oficina': '#C5C9C7',
 'Daviplata': '#6E750E',
 'AdministraciÃ³n de productos': '#DDA00D',
 'Soporte': '#FBDD7E',
 'Fallas Canales fÃ\xadsicos': '#E9967A',
 'otp': '#00FFFF',
 'solicitud_productos': '#7FFFD4',
 'chat': '#F5F5DC'}

In [12]:
warnings.filterwarnings('ignore')
final = pd.DataFrame(df.Categoria)

for i, col in enumerate(df.select_dtypes('float64').columns):
    ch1 = df[['Categoria', col]]
    ch1.sort_values(col, inplace=True)
    ch1[f'y{i}_upper'] = ch1[col].cumsum()
    ch1[f'y{i}_lower'] = ch1[f'y{i}_upper'].shift(1)
    ch1 = ch1.fillna(0)
    ch1[f'y{i}'] = ch1.apply(lambda x: (x[f'y{i}_upper']+x[f'y{i}_lower'])/2, axis=1)
    final = final.merge(ch1.iloc[:, [0, 2, 3, 4]], on='Categoria')

def getupperlower(Categoria):
    ch1 = final.query('Categoria == @Categoria')
    upper_col = [i for i in ch1.columns if 'upper' in i]
    lower_col = [i for i in ch1.columns if 'lower' in i]
    upper_data = ch1[upper_col].values.tolist()[0]
    lower_data = ch1[lower_col].values.tolist()[0]
    annotate_place = ch1['y0'].values.tolist()[0]
    return upper_data, lower_data, annotate_place

def get_val(x,sub=0.1, axis='x'):
    if axis != 'x':
        a = [[i, i]for i in x]
    else:
        a = [[i-sub, i+sub ]for i in x]
    return list(itertools.chain.from_iterable(a))

In [21]:
colors2 =[i for i in colors.values()]

fig = go.Figure()

# Ribbon
cant_periodos = len(df.select_dtypes('float64').columns)

# creating x axis value for the line chart
x = (np.arange(cant_periodos)+1).tolist()
x = get_val(x, sub=0.15)
x_rev = x[::-1]

annotations = []

for i, brand in enumerate(df.Categoria):

    upper_col, lower_col, annotate_place = getupperlower(brand)
    y_upper = upper_col
    y_upper = get_val(y_upper, axis="y")

    y_lower = lower_col
    y_lower = get_val(y_lower, axis="y")
    y_lower = y_lower[::-1]

    fig.add_trace(go.Scatter(
        x=x+[x[-1]+1, x[-1]+1]+x_rev,
        y=y_upper+[y_upper[-1], y_lower[0]]+y_lower,
        fill='toself',
        fillcolor=colors2[i],
        opacity=0.5,
        line_color='rgba(0,0,0,0.2)',
        showlegend=False,
        name=brand,
        line_shape='spline',
        mode='lines',#+markers
        marker=dict(size=20),
        hovertemplate=' '
    ))
    annotations.append(dict(xref='paper', yref='y',
                            x=-0.005, y=annotate_place,
                            text=brand,align="center",xanchor='right',
                            font=dict(family='Encode Sans Condensed', size=12, ## cambiar tamaño de letra del nombre de las etiquetas
                                      color=colors2[i]),
                            showarrow=False))

# bar chart
ch1 = df.set_index('Categoria')
for i,j in enumerate(df.select_dtypes('float64').columns):

    ch2 = pd.DataFrame(ch1.T.iloc[i])
    ch2.columns = [i+1]
    ch2.sort_values(i+1, ascending=True, inplace=True)
    fig_px = px.bar(ch2.T,  color_discrete_map=colors, opacity=0.7, text_auto=True)
    fig_px.update_traces(hovertemplate='<b>%{x}</b>, %{value}%')
    fig_px.update_traces(textfont=dict(size=12,color='black'), textposition='auto', cliponaxis=False, texttemplate='%{value}%') ### Size para modificar el dato de %motivo
    for trace in fig_px['data']:
        fig.add_trace(trace)



fig.update_layout(barmode='stack',  bargap=0.45, showlegend=False)

fig.update_layout(plot_bgcolor='#f2f3f4', paper_bgcolor='#FFFFFF', margin=dict(l=220,b=5,r=10,t=50)) ### Espacio de etiquetas
fig.update_xaxes(range=[0.90,9],tickmode = 'array',showticklabels=True, # que tan gorditas son las barras
            ticktext = df.select_dtypes('float64').columns,
            tickvals = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],fixedrange=True) ### Cambiar segun numero de meses a mostrar
fig.update_yaxes(range=[0,115],showticklabels=False, showgrid=False, fixedrange=True) ### Cambiar para que no se vea cortado
fig.update_layout(annotations=annotations)

fig.show()

In [ ]:
import matplotlib

for cname, hex in matplotlib.colors.cnames.items():
    print(cname, hex)

aliceblue #F0F8FF
antiquewhite #FAEBD7
aqua #00FFFF
aquamarine #7FFFD4
azure #F0FFFF
beige #F5F5DC
bisque #FFE4C4
black #000000
blanchedalmond #FFEBCD
blue #0000FF
blueviolet #8A2BE2
brown #A52A2A
burlywood #DEB887
cadetblue #5F9EA0
chartreuse #7FFF00
chocolate #D2691E
coral #FF7F50
cornflowerblue #6495ED
cornsilk #FFF8DC
crimson #DC143C
cyan #00FFFF
darkblue #00008B
darkcyan #008B8B
darkgoldenrod #B8860B
darkgray #A9A9A9
darkgreen #006400
darkgrey #A9A9A9
darkkhaki #BDB76B
darkmagenta #8B008B
darkolivegreen #556B2F
darkorange #FF8C00
darkorchid #9932CC
darkred #8B0000
darksalmon #E9967A
darkseagreen #8FBC8F
darkslateblue #483D8B
darkslategray #2F4F4F
darkslategrey #2F4F4F
darkturquoise #00CED1
darkviolet #9400D3
deeppink #FF1493
deepskyblue #00BFFF
dimgray #696969
dimgrey #696969
dodgerblue #1E90FF
firebrick #B22222
floralwhite #FFFAF0
forestgreen #228B22
fuchsia #FF00FF
gainsboro #DCDCDC
ghostwhite #F8F8FF
gold #FFD700
goldenrod #DAA520
gray #808080
green #008000
greenyellow #ADFF2F
